# 📊 PDF 데이터시트 전처리 도구

**데이터시트 PDF를 분석하여 유용한 정보를 추출하고 JSON 형태로 변환합니다.**

---

## 🚀 시작하기 전에

이 노트북은 PDF 데이터시트를 처리하여 나중에 챗봇에서 사용할 수 있는 JSON 파일을 생성합니다.
분할 방식으로 PDF를 처리하여 더 효율적으로 정보를 추출합니다.

## 📋 사용 방법
아래 셀들을 **순서대로 실행**만 하면 됩니다. (각 셀의 ▶️ 버튼을 클릭)

---

In [ ]:
#@title ✅ 1단계: Google Drive 연동 { display-mode: "form" }
#@markdown Google Drive를 연동하여 PDF 파일을 불러오고 처리된 JSON 파일과 벡터 스토어를 저장합니다.

from google.colab import drive
import os
from pathlib import Path

print("🔄 Google Drive 마운트 중...")
drive.mount('/content/drive')

# 폴더 경로 설정
pdf_folder = Path('/content/drive/MyDrive/datasheets')
result_json_folder = Path('/content/drive/MyDrive/prep_json')
vectorstore_folder = Path('/content/drive/MyDrive/vectorstore')

# 폴더가 없으면 생성
#pdf_folder.mkdir(parents=True, exist_ok=True)
result_json_folder.mkdir(parents=True, exist_ok=True)
vectorstore_folder.mkdir(parents=True, exist_ok=True)

print(f"\n✅ Google Drive 연동 완료!")
print(f"📁 PDF 폴더: {pdf_folder}")
print(f"📁 JSON 저장 폴더: {result_json_folder}")
print(f"📁 벡터 스토어 폴더: {vectorstore_folder}")

# PDF 파일 목록 확인
pdf_files = list(pdf_folder.glob("*.pdf"))
print(f"\n📄 발견된 PDF 파일: {len(pdf_files)}개")
for pdf in pdf_files[:5]:  # 처음 5개만 표시
    print(f" - {pdf.name}")
    
if len(pdf_files) > 5:
    print(f" ... 외 {len(pdf_files) - 5}개")

if not pdf_files:
    print("\n⚠️ PDF 파일이 없습니다. '{pdf_folder}' 폴더에 PDF 파일을 업로드해주세요.")

In [ ]:
#@title 📥 2단계: 필요한 라이브러리 설치 { display-mode: "form" }
#@markdown 필요한 라이브러리들을 설치합니다.

import os
from pathlib import Path

print("📥 필요한 라이브러리 설치 중...")

!pip install -r requirements.txt -q

# 현재 디렉토리 설정
import sys
current_dir = Path(os.getcwd())
sys.path.append(str(current_dir))
sys.path.append(str(current_dir / 'src'))

print("✅ 라이브러리 설치 완료!")

In [ ]:
#@title 🔑 3단계: API 키 설정 { display-mode: "form" }
#@markdown OpenAI API 키를 입력하여 PDF 분석을 위한 LLM을 설정합니다.

import os

openai_api_key = "" #@param {type:"string"}
claude_api_key = "" #@param {type:"string"}
hf_token = "" #@param {type:"string"}

# API 키 유효성 검사
if not openai_api_key and not claude_api_key and not hf_token:
    print("⚠️ API 키가 입력되지 않았습니다.")
    print("💡 최소 하나 이상의 API 키를 입력해주세요.")
    print("• OpenAI API 키: https://platform.openai.com/api-keys")
    print("• Claude API 키: https://console.anthropic.com/keys")
    print("• HuggingFace 토큰: https://huggingface.co/settings/tokens")
else:
    # 환경 변수 설정
    if openai_api_key:
        os.environ["OPENAI_API_KEY"] = openai_api_key
        print("✅ OpenAI API 키 설정 완료!")
        
    if claude_api_key:
        os.environ["ANTHROPIC_API_KEY"] = claude_api_key
        print("✅ Claude API 키 설정 완료!")
    
    if hf_token:
        os.environ["HF_TOKEN"] = hf_token
        print("✅ HuggingFace 토큰 설정 완료!")

In [ ]:
#@title 📊 4단계: PDF 파일 처리 설정 { display-mode: "form" }
#@markdown PDF 처리 설정을 구성합니다.

# 청크 크기 설정 (한 번에 처리할 페이지 수)
chunk_size = 3 #@param {type:"slider", min:1, max:10, step:1}
provider = "openai" #@param ["openai", "claude"]

print(f"📝 PDF 처리 설정:")
print(f" - 청크 크기: {chunk_size} 페이지")
print(f" - LLM 제공자: {provider}")

# API 키 확인
if provider == "openai" and "OPENAI_API_KEY" not in os.environ:
    print("❌ OpenAI API 키가 설정되지 않았습니다. 3단계를 먼저 완료해주세요.")
elif provider == "claude" and "ANTHROPIC_API_KEY" not in os.environ:
    print("❌ Claude API 키가 설정되지 않았습니다. 3단계를 먼저 완료해주세요.")
else:
    print("✅ 설정이 완료되었습니다.")

In [ ]:
#@title 📄 5단계: PDF 파일 처리 (단일 파일) { display-mode: "form" }
#@markdown 선택한 PDF 파일을 처리합니다.

from src.pdf_processor import PDFProcessor
import json
import os

# 필요한 API 키 설정 확인
api_key = os.environ.get("OPENAI_API_KEY" if provider == "openai" else "ANTHROPIC_API_KEY", "")
claude_api_key = os.environ.get("ANTHROPIC_API_KEY", "") if provider == "claude" else None

if not api_key:
    print(f"❌ {provider.capitalize()} API 키가 설정되지 않았습니다. 3단계를 먼저 완료해주세요.")
else:
    # PDF 처리기 초기화
    processor = PDFProcessor(api_key, provider, claude_api_key, chunk_size)
    
    # 처리할 PDF 파일 선택
    if not pdf_files:
        print("❌ 처리할 PDF 파일이 없습니다. Google Drive에 파일을 업로드해주세요.")
    else:
        print("📝 처리할 PDF 파일 목록:")
        for i, pdf in enumerate(pdf_files):
            print(f"{i+1}. {pdf.name}")
            
        # PDF 인덱스 입력 (1부터 시작)
        pdf_index = 1  #@param {type:"integer"}
        
        if pdf_index < 1 or pdf_index > len(pdf_files):
            print(f"❌ 잘못된 인덱스입니다. 1에서 {len(pdf_files)} 사이의 값을 입력하세요.")
        else:
            # 선택한 PDF 처리
            selected_pdf = pdf_files[pdf_index-1]
            print(f"\n🔍 '{selected_pdf.name}' 파일을 처리합니다...")
            print(f"⚙️ 처리 방식: 청크 크기 {chunk_size} 페이지로 분할 처리")
            print("⏳ 이 작업은 몇 분 정도 걸릴 수 있습니다. 잠시만 기다려주세요...")
            
            try:
                features, useful_pages = processor.process_pdf(str(selected_pdf))
                
                # 결과 저장
                if features:
                    output_file = result_json_folder / f"{selected_pdf.stem}.json"
                    with open(output_file, 'w', encoding='utf-8') as f:
                        json.dump(features, f, indent=2, ensure_ascii=False)
                        
                    print(f"\n✅ 처리 완료!")
                    print(f"📄 추출된 유용한 페이지: {useful_pages}")
                    print(f"💾 결과가 저장되었습니다: {output_file}")
                    print("\n📊 추출된 정보 미리보기:")
                    print(json.dumps(features, indent=2, ensure_ascii=False)[:500] + '...')
                else:
                    print("❌ 유용한 정보를 추출하지 못했습니다.")
            except Exception as e:
                print(f"❌ 처리 중 오류가 발생했습니다: {str(e)}")

In [ ]:
#@title 🔄 6단계: 모든 PDF 파일 일괄 처리 { display-mode: "form" }
#@markdown 모든 PDF 파일을 한 번에 처리합니다. (시간이 오래 걸릴 수 있습니다)

import os
import json
from tqdm.notebook import tqdm

# 필요한 API 키 설정 확인
api_key = os.environ.get("OPENAI_API_KEY" if provider == "openai" else "ANTHROPIC_API_KEY", "")
claude_api_key = os.environ.get("ANTHROPIC_API_KEY", "") if provider == "claude" else None

if not api_key:
    print(f"❌ {provider.capitalize()} API 키가 설정되지 않았습니다. 3단계를 먼저 완료해주세요.")
else:
    # PDF 처리기 초기화
    processor = PDFProcessor(api_key, provider, claude_api_key, chunk_size)
    
    # 처리할 PDF 파일 확인
    if not pdf_files:
        print("❌ 처리할 PDF 파일이 없습니다. Google Drive에 파일을 업로드해주세요.")
    else:
        # 일괄 처리 여부 확인
        confirm = True  #@param {type:"boolean"}
        
        if confirm:
            print(f"🔄 {len(pdf_files)}개의 PDF 파일을 처리합니다...")
            print(f"⚙️ 처리 방식: 청크 크기 {chunk_size} 페이지로 분할 처리")
            print("⏳ 이 작업은 시간이 오래 걸릴 수 있습니다. 잠시만 기다려주세요...")
            
            # 처리 결과 저장
            results = {}
            
            # tqdm으로 진행 상황 표시
            for pdf in tqdm(pdf_files, desc="PDF 처리 중"):
                try:
                    features, useful_pages = processor.process_pdf(str(pdf))
                    
                    if features:
                        # 결과 저장
                        output_file = result_json_folder / f"{pdf.stem}.json"
                        with open(output_file, 'w', encoding='utf-8') as f:
                            json.dump(features, f, indent=2, ensure_ascii=False)
                        
                        results[pdf.name] = {
                            "status": "성공",
                            "useful_pages": useful_pages,
                            "output_file": str(output_file)
                        }
                    else:
                        results[pdf.name] = {
                            "status": "실패",
                            "reason": "유용한 정보를 추출하지 못했습니다."
                        }
                except Exception as e:
                    results[pdf.name] = {
                        "status": "오류",
                        "reason": str(e)
                    }
            
            # 결과 요약
            success_count = sum(1 for result in results.values() if result["status"] == "성공")
            failed_count = len(results) - success_count
            
            print(f"\n✅ 처리 완료! 총 {len(results)}개 중 {success_count}개 성공, {failed_count}개 실패")
            print(f"💾 결과가 '{result_json_folder}'에 저장되었습니다.")
            
            if failed_count > 0:
                print("\n⚠️ 실패한 파일:")
                for name, result in results.items():
                    if result["status"] != "성공":
                        print(f" - {name}: {result.get('reason', '알 수 없는 오류')}")
        else:
            print("❌ 일괄 처리가 취소되었습니다.")

In [ ]:
#@title 🔄 7단계: 벡터 스토어 생성 { display-mode: "form" }
#@markdown 처리된 JSON 파일로부터 벡터 스토어를 생성합니다.

from src.vectorstore_manager import VectorstoreManager
import os

# HF 토큰 가져오기
hf_token = os.environ.get("HF_TOKEN", "")
if not hf_token:
    print("❌ HuggingFace 토큰이 설정되지 않았습니다. 3단계를 먼저 완료해주세요.")
else:
    # 벡터 스토어 매니저 초기화
    vectorstore_manager = VectorstoreManager(hf_token=hf_token)
    
    # JSON 파일 확인
    if not result_json_folder.exists():
        print("❌ JSON 폴더가 존재하지 않습니다.")
    else:
        # JSON 파일 로드
        try:
            json_data = vectorstore_manager.load_json_files(str(result_json_folder))
            
            if not json_data:
                print("❌ JSON 파일이 없습니다. 먼저 PDF 파일을 처리해주세요.")
            else:
                print(f"✅ {len(json_data)}개의 JSON 파일을 로드했습니다.")
                print("\n🔄 벡터 스토어를 생성 중입니다...")
                
                # 벡터 스토어 생성
                vectorstore = vectorstore_manager.create_vectorstore(json_data)
                
                # 벡터 스토어 저장
                print("\n💾 벡터 스토어를 저장 중입니다...")
                vectorstore_manager.save_vectorstore(vectorstore, str(vectorstore_folder))
                
                print(f"\n✅ 벡터 스토어가 성공적으로 생성되었습니다!")
                print(f"📁 저장 위치: {vectorstore_folder}")
        except Exception as e:
            print(f"❌ 벡터 스토어 생성 중 오류가 발생했습니다: {str(e)}")

## 🎉 처리 완료!

PDF 파일 처리와 벡터 스토어 생성이 완료되었습니다.

- 처리된 JSON 파일은 Google Drive의 `prep_json` 폴더에 저장되었습니다.
- 벡터 스토어는 Google Drive의 `vectorstore` 폴더에 저장되었습니다.

이제 ChipChat 모듈에서 이 데이터를 불러와 데이터시트에 대한 질의응답을 수행할 수 있습니다.